# 10 - EVALUACIÓN CARACTERIZACION Clusters DBSCAN

**AUTOR: Fabrizio Ramirez Cutimbo**

**OBJETIVO**

+ Aplicar tecnicas de Clustering DBSCAN a los dataset preprocesados.
+ Evaluar la mejor configuración de Clusters en relación al epsilon y número mínimo de elementos
+ Generar clusters con la mejor configuración posible
+ Visualizar los clusters generados.
---

In [194]:
# Librerias de analisis de datos
import pandas as pd
import numpy as np

# Graficos
import matplotlib.pyplot as plt
import seaborn as sns

In [195]:
# Establecere formato de 2 decimales
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None) 

pd_companias = pd.read_csv('Datasets Procesados/directorio_empresas_final_clusterizado.csv', dtype={'RUC': 'object'})

In [196]:
print(pd_companias.shape)
pd_companias.sample(2)

(37884, 77)


,EXPEDIENTE,RUC,SITUACION_LEGAL,FECHA_CONSTITUCION,TIPO,PAIS,REGION,PROVINCIA,CANTON,CIUDAD,...,IF_MARGEN_OPERCIONAL,IF_MARGEN_NETO,IF_ROE,IF_ROA,Set10_PCA1,Set10_PCA2,CLUSTERS_KMEANS_SET_10,Set4_UMAP1,Set4_UMAP2,CLUSTER_DBSCAN_SET_4
23012,336395,0993373250001,ACTIVA,29/06/2022,SOCIEDAD POR ACCIONES SIMPLIFICADA,ECUADOR,COSTA,GUAYAS,GUAYAQUIL,GUAYAQUIL,...,0.01,0.00,0.00,0.00,0.60,-0.60,3,2.51,18.37,6
33276,731482,0993284475001,ACTIVA,22/09/2020,SOCIEDAD POR ACCIONES SIMPLIFICADA,ECUADOR,COSTA,GUAYAS,GUAYAQUIL,GUAYAQUIL,...,0.04,0.04,0.41,0.08,0.53,-0.76,3,9.18,5.91,28


In [197]:
# pd_companias.columns

In [198]:
# Reemplazar valores por fomato correcto
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('MICRO', 'Micro')
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('PEQUENA', 'Pequeña')
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('MEDIANA', 'Mediana')
pd_companias['SEGMENTO'] = pd_companias['SEGMENTO'].str.replace('GRANDE', 'Grande')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.capitalize()
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Galapagos', 'Galápagos')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Los rios', 'Los ríos')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Morona santiago', 'Morona Santiago')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Santo domingo de los tsachilas', 'Santo Domingo de los Tsáchilas')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Santa elena', 'Santa Elena')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Zamora chinchipe', 'Zamora Chinchipe')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('Sucumbios', 'Sucumbíos')
pd_companias['PROVINCIA'] = pd_companias['PROVINCIA'].str.replace('El oro', 'El Oro')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('COSTA', 'Costa')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('SIERRA', 'Sierra')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('AMAZONIA', 'Amazonía')
pd_companias['REGION'] = pd_companias['REGION'].str.replace('GALÁPAGOS', 'Galápagos')

### Tabla resumen clusters DBSCAN

#### Funciones

In [199]:
def agrupar_por_cluster(dataset, columna_agrupacion):
    """
    Agrupa un dataset por una columna específica y calcula promedios, distribuciones
    de segmentos, regiones y actividades económicas.

    Args:
        dataset (pd.DataFrame): El conjunto de datos a agrupar.
        columna_agrupacion (str): La columna por la cual se agrupará el dataset. El Nombre del Cluster.

    Returns:
        pd.DataFrame: Un DataFrame con los resultados del análisis por cluster.
    """

    # Agrupar por la columna especificada
    grouped_clusters = dataset.groupby(columna_agrupacion)

    # Crear lista para almacenar los resultados
    results = []

    # Iterar sobre cada grupo
    for cluster, group in grouped_clusters:

        #Contar el numero de empresas
        n_empresas = group.shape[0] 
        
        acid_test_stats = group['IF_PRUEBA_ACIDA'].agg(['mean', 'std', 'min', 'median', 'max'])
        acid_test_summary = (
            f"Mean: {acid_test_stats['mean']:.1f}, "
            # f"Std: {acid_test_stats['std']:.1f}, "            
            f"Median: {acid_test_stats['median']:.1f} "            
        )

        if_roe_stats = group['IF_ROE'].agg(['mean', 'std', 'min', 'median', 'max'])
        if_roe_summary = (
            #multiplicar por 100 y poner %
            f"Mean: {if_roe_stats['mean']*100:.1f}%, "
            # f"Mean: {if_roe_stats['mean']:.1f}, "
            # f"Std: {if_roe_stats['std']:.1f}, "            
            f"Median: {if_roe_stats['median']*100:.1f}% "            
        )
        if_roa_stats = group['IF_ROA'].agg(['mean', 'std', 'min', 'median', 'max'])
        if_roa_summary = (
            f"Mean: {if_roa_stats['mean']*100:.1f}%, "
            # f"Std: {if_roa_stats['std']:.1f}, "            
            f"Median: {if_roa_stats['median']*100:.1f}% "            
        )
        if_mg_neto_stats = group['IF_MARGEN_NETO'].agg(['mean', 'std', 'min', 'median', 'max'])
        if_mg_neto_summary = (
            f"Mean: {if_mg_neto_stats['mean']*100:.1f}%, "
            # f"Std: {if_mg_neto_stats['std']:.1f}, "            
            f"Median: {if_mg_neto_stats['median']*100:.1f}% "            
        )

        ingresos_stats = group['INGRESOS_ACTIVIDADES_ORDINARIAS_2023'].agg(['mean', 'std', 'min', 'median', 'max'])
        ingresos_summary = (
            f"Mean: ${ingresos_stats['mean']:,.1f}, "
            # f"Std: {ingresos_stats['std']:.1f} "
            f"Median: ${ingresos_stats['median']:,.1f}, "
        )        

        utilidad_neta_stats = group['UTILIDAD_NETA_2023'].agg(['mean', 'std', 'min', 'median', 'max'])
        utilidad_neta_summary = (
            f"Mean: ${utilidad_neta_stats['mean']:,.1f}, "
            # f"Std: {utilidad_neta_stats['std']:.1f}, "
            # f"Min: ${utilidad_neta_stats['min']:.1f}, "
            f"Median: ${utilidad_neta_stats['median']:,.1f} "
            # f"Max: ${utilidad_neta_stats['max']:.2f}"
        )         
        

        # Calcular distribución del segmento
        segment_distribution = group['SEGMENTO'].value_counts(normalize=True) * 100
        segment_distribution_str = ', '.join([f"{seg}: {perc:.2f}%" for seg, perc in segment_distribution.items()])

        # # Calcular distribución de la región
        # region_distribution = group['REGION'].value_counts(normalize=True) * 100
        # region_distribution_str = ', '.join([f"{reg}: {perc:.2f}%" for reg, perc in region_distribution.items()])

        # Calcular distribución de la Provincia
        provincia_distribution = group['PROVINCIA'].value_counts(normalize=True) * 100
        # provincia_distribution_str = ', '.join([f"{reg}: {perc:.2f}%" for reg, perc in provincia_distribution.items()]) # Considerar todos los elementos
        provincia_distribution_str = ', '.join([f"{reg}: {perc:.2f}%" for reg, perc in provincia_distribution.head(3).items()])
        if len(provincia_distribution) > 3:
            provincia_distribution_str += ", ..."

        # Calcular distribución de CIIU_NIVEL_1
        ciiu_distribution = group['CIIU_NIVEL_1'].value_counts(normalize=True) * 100
        ciiu_distribution_str = ', '.join([f"{ciiu}: {perc:.2f}%" for ciiu, perc in ciiu_distribution.items()])

        # Almacenar resultados en la lista
        result_row = {
            columna_agrupacion: cluster,
            'Acid Test': acid_test_summary,
            'ROE': if_roe_summary,
            'ROA': if_roa_summary,
            'Margen Neto': if_mg_neto_summary,
            'Ingresos': ingresos_summary,
            'Utilidad Neta': utilidad_neta_summary,
            'Nro. Empresas': n_empresas,
            'Distribución Segmento': segment_distribution_str,
            # 'Distribución Región': region_distribution_str,
            'Distribución Provincia': provincia_distribution_str,
            'Distribución CIIU 1': ciiu_distribution_str
        }

        results.append(result_row)

    # Convertir resultados a DataFrame
    result_df = pd.DataFrame(results)

    # Reorganizar las columnas para que las distribuciones se muestren al final
    # columnas_distribucion = ['Distribución Segmento', 'Distribución Región', 'Distribución CIIU 1']
    columnas_distribucion = ['Distribución Segmento', 'Distribución Provincia', 'Distribución CIIU 1']
    columnas_ordenadas = (
        [columna_agrupacion] +
        [col for col in result_df.columns if col not in columnas_distribucion and col != columna_agrupacion] +
        columnas_distribucion
    )

    # Reorganizar DataFrame
    result_df = result_df[columnas_ordenadas]

    return result_df

# Llamar a la función y mostrar el DataFrame resultante
result_df = agrupar_por_cluster(pd_companias, 'CLUSTER_DBSCAN_SET_4')

# Renombrar la columna de Cluster
result_df.rename(columns={'CLUSTER_DBSCAN_SET_4': 'Cluster'}, inplace=True)

# Excluir el Cluster -1 de Empreas No Agrupadas
result_df = result_df[result_df['Cluster'] != -1]

result_df


,Cluster,Acid Test,ROE,ROA,Margen Neto,Ingresos,Utilidad Neta,Nro. Empresas,Distribución Segmento,Distribución Provincia,Distribución CIIU 1
1,0,"Mean: 1.6, Median: 1.0","Mean: 6.5%, Median: 0.0%","Mean: 2.3%, Median: 0.0%","Mean: 2.0%, Median: 0.0%","Mean: $886,212.1, Median: $286,714.1,","Mean: $8,865.3, Median: $0.0",666,"Pequeña: 44.29%, Micro: 28.68%, Mediana: 26.58%, Grande: 0.45%","Pichincha: 56.46%, Santo Domingo de los Tsáchilas: 9.61%, Cotopaxi: 8.71%, ...","A: 98.35%, N: 0.60%, M: 0.60%, J: 0.30%, C: 0.15%"
2,1,"Mean: 1.3, Median: 1.0","Mean: 10.0%, Median: 4.9%","Mean: 4.1%, Median: 1.8%","Mean: 3.4%, Median: 1.5%","Mean: $13,693,737.9, Median: $15,389,331.7,","Mean: $64,933.6, Median: $122,394.4",307,Grande: 100.00%,"Pichincha: 70.03%, Azuay: 13.03%, Tungurahua: 7.17%, ...",C: 100.00%
3,2,"Mean: 1.4, Median: 1.0","Mean: 7.6%, Median: 0.7%","Mean: 2.7%, Median: 0.2%","Mean: 2.0%, Median: 0.2%","Mean: $2,391,803.8, Median: $2,071,395.3,","Mean: $31,425.5, Median: $3,676.6",478,"Mediana: 99.79%, Grande: 0.21%","Pichincha: 69.25%, Azuay: 13.18%, Tungurahua: 6.90%, ...",C: 100.00%
4,3,"Mean: 1.3, Median: 0.9","Mean: 17.2%, Median: 0.0%","Mean: 1.1%, Median: 0.0%","Mean: 0.9%, Median: 0.0%","Mean: $543,089.2, Median: $165,798.4,","Mean: $3,589.3, Median: $0.0",2037,"Pequeña: 49.58%, Micro: 40.30%, Mediana: 8.59%, Grande: 1.52%","Guayas: 37.65%, Pichincha: 34.61%, Azuay: 6.14%, ...","I: 53.71%, A: 12.08%, N: 11.29%, C: 10.51%, J: 8.89%, M: 2.26%, K: 1.28%"
5,4,"Mean: 1.3, Median: 1.0","Mean: 10.0%, Median: 1.4%","Mean: 2.8%, Median: 0.3%","Mean: 1.7%, Median: 0.1%","Mean: $3,958,223.3, Median: $1,904,062.4,","Mean: $26,799.8, Median: $1,346.6",137,"Mediana: 75.91%, Grande: 19.71%, Pequeña: 4.38%","Pichincha: 70.80%, Azuay: 10.95%, Sucumbíos: 3.65%, ...",I: 100.00%
...,...,...,...,...,...,...,...,...,...,...,...
57,56,"Mean: 1.0, Median: 1.0","Mean: -0.1%, Median: 0.0%","Mean: -0.1%, Median: 0.0%","Mean: 60.5%, Median: 60.5%","Mean: $3.5, Median: $0.0,","Mean: $234.3, Median: $0.0",128,Micro: 100.00%,"Guayas: 75.00%, Manabi: 11.72%, El Oro: 5.47%, ...",N: 100.00%
58,57,"Mean: 21.7, Median: 23.5","Mean: 1.3%, Median: 0.0%","Mean: 0.5%, Median: 0.0%","Mean: 60.1%, Median: 60.5%","Mean: $1,271.0, Median: $0.0,","Mean: $704.8, Median: $0.0",136,Micro: 100.00%,"Guayas: 78.68%, El Oro: 8.82%, Manabi: 5.15%, ...","M: 53.68%, C: 18.38%, N: 13.24%, J: 10.29%, I: 4.41%"
59,58,"Mean: 2.1, Median: 1.2","Mean: -1.7%, Median: 0.0%","Mean: -0.7%, Median: 0.0%","Mean: 60.5%, Median: 60.5%","Mean: $740.9, Median: $0.0,","Mean: $-357.8, Median: $0.0",111,Micro: 100.00%,"Guayas: 90.09%, Manabi: 6.31%, Santa Elena: 1.80%, ...",M: 100.00%
60,59,"Mean: 0.7, Median: 0.9","Mean: -12.9%, Median: 0.0%","Mean: -1.0%, Median: 0.0%","Mean: 60.2%, Median: 60.5%","Mean: $148.3, Median: $0.0,","Mean: $-3,615.6, Median: $0.0",159,Micro: 100.00%,"Guayas: 42.77%, Pichincha: 33.33%, El Oro: 4.40%, ...","A: 36.48%, M: 23.90%, K: 12.58%, N: 10.06%, C: 6.29%, I: 5.66%, J: 5.03%"


#### Resumen General de Evaluación:
- Acid Test: Liquidez a corto plazo, idealmente > 1.
- ROE: Retorno para accionistas, > 15% es excelente.
- ROA: Eficiencia en uso de activos, > 10% refleja buena gestión.
- Margen Neto: Rentabilidad final, > 20% indica eficiencia y control de costos.

Exportar tabla resumen

In [200]:
result_df.to_csv(
    'Datasets Procesados\\Resultados\\Tabla_Resumen_Clusters_dbscan.csv', 
    sep=';',               # Separador de columnas
    decimal=',',           # Separador decimal
    index=False,           # Excluir índice
    encoding='utf-8-sig'   # Codificación compatible con Excel y Power BI
)

## Evaluación Location Quotient (LQ)

In [201]:
def calcular_location_quotient(dataset, metrica, provincia_col, cluster_col):
    """
    Calcula el Quotient Location (QL) para un dataset dado.

    Parámetros:
    - dataset: DataFrame de entrada que contiene los datos.
    - metrica: Columna que representa la métrica económica (e.g., Empleados UTILIDAD_NETA_2023).
    - provincia_col: Nombre de la columna que contiene las provincias (o región Menor).
    - cluster_col: Nombre de la columna que contiene los clusters (La Actividad Económica).

    Retorna:
    - DataFrame con columnas para provincia, cluster y el valor calculado de QL.
    """
    # Calcular totales por cluster y provincia
    # totales_clusters_provincias = dataset.groupby([provincia_col, cluster_col])[metrica].sum()
    totales_clusters_provincias = dataset.groupby([provincia_col, cluster_col])[metrica].agg(['sum', 'count']).reset_index()

    # Calcular totales por provincia
    totales_provincias = dataset.groupby(provincia_col)[metrica].sum()

    # Calcular totales por cluster a nivel nacional
    totales_clusters_nacional = dataset.groupby(cluster_col)[metrica].sum()

    # Calcular total nacional
    totales_nacional = dataset[metrica].sum()

    # Crear lista para almacenar los resultados
    lq_data = []
    
    for index, row in totales_clusters_provincias.iterrows():
        provincia = row[provincia_col]
        cluster = row[cluster_col]
        total_suma = row['sum']
        total_conteo = row['count']
        # Proporción del cluster dentro de la provincia
        proporcion_cluster_provincia = total_suma / totales_provincias[provincia]

        # Proporción del cluster a nivel nacional
        proporcion_cluster_region = totales_clusters_nacional[cluster] / totales_nacional

        # Se ha detectado que hay empresas que a pesar de tener ingresos no tienen resultados del ejercicio (utilidad, impuestos, etc)
        # Estos valores se establecen en 0, auqnue podrían establecerse en NaN para análisis futuros
        if proporcion_cluster_region == 0:
            lq = 0 # Si el Divisor es 0, no se realiza la división sino se establece directamente en 0
            
        else: 
            # Calcular QL
            lq = proporcion_cluster_provincia / proporcion_cluster_region

        # # Verificar si alguna proporción es un valor no numérico
        # if not np.isfinite(lq):
        #     print(f"Valor no numérico encontrado: {provincia} - {cluster}")
        #     print(f"Proporciones: {proporcion_cluster_provincia}, {proporcion_cluster_region}")
        #     lq = np.nan        

        # Agregar resultado a la lista
        lq_data.append({
            provincia_col: provincia,
            cluster_col: cluster,
            'Nro. Empresas': total_conteo,
            'LQ': lq
        })

    # Convertir resultados a DataFrame
    lq_dataframe = pd.DataFrame(lq_data)

    # Ordenar por QL descendente
    lq_dataframe = lq_dataframe.sort_values(by='LQ', ascending=False)

    return lq_dataframe

Calculo de LQ para CLUSTER_DBSCAN_SET_4

In [202]:
location_quotient_set_4_dbscan = calcular_location_quotient(dataset=pd_companias, 
                                                            metrica='INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 
                                                            provincia_col='PROVINCIA', 
                                                            cluster_col='CLUSTER_DBSCAN_SET_4')

print(f'Combinaciones Provinca x CLUSTER_DBSCAN_SET_4: {location_quotient_set_4_dbscan.shape[0]}')

# Filtros de Clusters
numero_minimo_empresas = 20
lq_altamente_conentrado = 3
print(f'Número mínimo de empresas: {numero_minimo_empresas}')
print(f'LQ Altamente Concentrado: {lq_altamente_conentrado}')

location_quotient_set_4_dbscan = location_quotient_set_4_dbscan[(location_quotient_set_4_dbscan['Nro. Empresas'] >= numero_minimo_empresas) & 
                                                                          (location_quotient_set_4_dbscan['LQ'] >= lq_altamente_conentrado) &
                                                                          (location_quotient_set_4_dbscan['CLUSTER_DBSCAN_SET_4'] != -1)]
print(f'Clusters Empresariales Identificados: {location_quotient_set_4_dbscan.shape[0]}')

location_quotient_set_4_dbscan = location_quotient_set_4_dbscan.sort_values(by=['PROVINCIA', 'LQ'], ascending=[True, True])

location_quotient_set_4_dbscan.head(20)


Combinaciones Provinca x CLUSTER_DBSCAN_SET_4: 757
Número mínimo de empresas: 20
LQ Altamente Concentrado: 3
Clusters Empresariales Identificados: 28


,PROVINCIA,CLUSTER_DBSCAN_SET_4,Nro. Empresas,LQ
6,Azuay,5,152,3.04
15,Azuay,17,35,3.07
3,Azuay,2,63,3.41
2,Azuay,1,40,3.47
122,Chimborazo,11,48,15.51
146,Cotopaxi,0,58,9.38
187,El Oro,10,63,3.56
194,El Oro,24,173,4.18
186,El Oro,9,54,4.88
201,El Oro,32,124,5.24


### Generación Tabla Resumen Aglomeraciones por LQ

In [203]:
# Realizamos el MERGE entre los dos datasets por las columnas 'Provincia' y 'CIIU_NIVEL_4_DESC'
merged_df = pd.merge(location_quotient_set_4_dbscan,
                     pd_companias[['RUC', 'PROVINCIA', 'CLUSTER_DBSCAN_SET_4', 'CIIU_NIVEL_6_DESC', 'SEGMENTO', 'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'UTILIDAD_NETA_2023']],                       
                     on=['PROVINCIA', 'CLUSTER_DBSCAN_SET_4'], 
                     how='inner')


grouped_df = merged_df.groupby(['PROVINCIA', 'CLUSTER_DBSCAN_SET_4', 'Nro. Empresas', 'LQ'], 
                               as_index=False)[['INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 
                                                'UTILIDAD_NETA_2023']].mean()

grouped_df = grouped_df.rename(columns={'INGRESOS_ACTIVIDADES_ORDINARIAS_2023': 'INGRESOS PROMEDIO',
                                        'UTILIDAD_NETA_2023': 'UTILIDAD PROMEDIO'})

grouped_df = grouped_df[grouped_df['UTILIDAD PROMEDIO'] > 0]

grouped_df.shape
grouped_df

,PROVINCIA,CLUSTER_DBSCAN_SET_4,Nro. Empresas,LQ,INGRESOS PROMEDIO,UTILIDAD PROMEDIO
0,Azuay,1,40,3.47,13801852.73,80860.65
1,Azuay,2,63,3.41,2343409.48,24945.98
2,Azuay,5,152,3.04,295417.84,10411.85
3,Azuay,17,35,3.07,1228822.15,91971.30
4,Chimborazo,11,48,15.51,266314.28,7668.31
5,Cotopaxi,0,58,9.38,1208418.16,13521.81
8,El Oro,24,173,4.18,506490.77,6137.80
9,El Oro,32,124,5.24,2394855.16,17524.14
11,Galápagos,3,35,3.37,549905.13,5728.10
12,Galápagos,31,96,24.75,1270977.78,15353.87


In [204]:
grouped_df.to_csv(
    'Datasets Procesados\\Resultados\\LQ_Clusters_dbscan.csv', 
    sep=';',               # Separador de columnas
    decimal=',',           # Separador decimal
    index=False,           # Excluir índice
    encoding='utf-8-sig'   # Codificación compatible con Excel y Power BI
)

In [205]:
# # Realizamos el MERGE entre los dos datasets por las columnas 'Provincia' y 'CIIU_NIVEL_4_DESC'
# merged_df = pd.merge(location_quotient_set_4_dbscan,
#                      pd_companias[['RUC', 'PROVINCIA', 'CLUSTER_DBSCAN_SET_4', 'CIIU_NIVEL_6_DESC', 'SEGMENTO', 'INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 'UTILIDAD_NETA_2023']],                       
#                      on=['PROVINCIA', 'CLUSTER_DBSCAN_SET_4'], 
#                      how='inner')


# grouped_df = merged_df.groupby(['PROVINCIA', 'CLUSTER_DBSCAN_SET_4', 'Nro Empresas', 'LQ'], 
#                                as_index=False)[['INGRESOS_ACTIVIDADES_ORDINARIAS_2023', 
#                                                 'UTILIDAD_NETA_2023']].mean()

# grouped_df = grouped_df.rename(columns={'INGRESOS_ACTIVIDADES_ORDINARIAS_2023': 'INGRESOS PROMEDIO',
#                                         'UTILIDAD_NETA_2023': 'UTILIDAD PROMEDIO'})

# grouped_df = grouped_df[grouped_df['UTILIDAD PROMEDIO'] > 0]

# Capitalizar columna CIIU_NIVEL_3_DESC de dataset pd_companias
pd_companias['CIIU_NIVEL_3_DESC'] = pd_companias['CIIU_NIVEL_3_DESC'].str.capitalize()

# Iterar cada uno de los registros
for index, row in grouped_df.iterrows():

    # Filtrar por PROVINCIA y CLUSTER_DBSCAN_SET4
    df_filtrado = pd_companias[(pd_companias['PROVINCIA'] == row['PROVINCIA']) & (pd_companias['CLUSTER_DBSCAN_SET_4'] == row['CLUSTER_DBSCAN_SET_4'])]

    total_empresas = df_filtrado.shape[0] # Total Filas

     # Calcular distribución del segmento
    segment_distribution = df_filtrado['SEGMENTO'].value_counts(normalize=True) * 100
    segment_distribution_str = ', '.join([f"{seg}: {perc:.2f}%" for seg, perc in segment_distribution.items()])

    grouped_df.loc[index, 'Distribución 2 SEGMENTO'] = segment_distribution_str


    # Calcular distribución del CIIU Nivel 1
    ciiu_n1_distribution = df_filtrado['CIIU_NIVEL_1'].value_counts(normalize=True) * 100
    ciiu_n1_distribution_str = ', '.join([f"{seg}: {perc:.2f}%" for seg, perc in ciiu_n1_distribution.items()])

    grouped_df.loc[index, 'Distribución CIIU1 '] = ciiu_n1_distribution_str

    # # Agrupar por CIIU NIVEL 3
    pd_ciiu_detalle = df_filtrado.groupby(['PROVINCIA', 'CLUSTER_DBSCAN_SET_4', 'CIIU_NIVEL_3_DESC']).size().reset_index(name='TOTAL')    
    #orderna decreciente
    pd_ciiu_detalle = pd_ciiu_detalle.sort_values(by='TOTAL', ascending=False)
    # Concatenar los valores de 'CIIU_NIVEL_6_DESC' con el 'TOTAL' y separarlos por comas
    ciiu_detalle_concat = pd_ciiu_detalle.apply(lambda x: f"{x['CIIU_NIVEL_3_DESC']} ({x['TOTAL']})", axis=1).tolist()    
    # Unir todos los valores con comas
    ciiu_detalle_string = ", ".join(ciiu_detalle_concat)
    # Añadir la columna "CIIU_NIVEL_6_DESC_lista" al dataframe
    grouped_df.loc[index, 'CIIU_NIVEL_3_DESC_lista'] = ciiu_detalle_string

   
    # print(pd_ciiu_detalle[['CIIU_NIVEL_6_DESC', 'TOTAL']])

# Formato Dolares
grouped_df['INGRESOS PROMEDIO'] = grouped_df['INGRESOS PROMEDIO'].apply(lambda x: f"${x:,.2f}")
grouped_df['UTILIDAD PROMEDIO'] = grouped_df['UTILIDAD PROMEDIO'].apply(lambda x: f"${x:,.2f}")
grouped_df['LQ'] = grouped_df['LQ'].apply(lambda x: f"{x:,.1f}")

# Renombrar columnas
grouped_df.rename(columns={'PROVINCIA': 'Provincia',
                           'CLUSTER_DBSCAN_SET_4': 'Cluster',
                           'Distribución 2 SEGMENTO': 'Distribución Segmento',
                           'Distribución CIIU1 ': 'Distribución CIIU 1',
                           'CIIU_NIVEL_3_DESC_lista': 'Distribución CIIU 3',
                           'INGRESOS PROMEDIO': 'Ingresos Promedio',
                           'UTILIDAD PROMEDIO': 'Utilidad Promedio'}, inplace=True)


grouped_df.sample(4)

,Provincia,Cluster,Nro. Empresas,LQ,Ingresos Promedio,Utilidad Promedio,Distribución Segmento,Distribución CIIU 1,Distribución CIIU 3
26,Tungurahua,2,33,3.6,"$2,427,999.38","$40,791.29",Mediana: 100.00%,C: 100.00%,"Fabricación de prendas de vestir, excepto prendas de piel (5), Elaboración de alimentos preparados para animales (4), Hilatura, tejedura y acabados de productos textiles (3), Fabricación de productos de plástico (2), Curtido y adobo de cueros; fabricación de maletas, bolsos de mano y artículos de talabartería y guarnicionería; adobo y teñido de pieles (2), Fabricación de productos de caucho (2), Fabricación de artículos de punto y ganchillo (2), Fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos de uso farmacéutico (2), Elaboración de bebidas (1), Fabricación de calzado (1), Elaboración de productos de molinería, almidones y productos derivados del almidón (1), Fabricación de otros productos textiles (1), Fabricación de otros productos químicos (1), Fabricación de hojas de madera para enchapado y tableros a base de madera (1), Fabricación de carrocerías para vehículos automotores; fabricación de remolques y semirremolques (1), Fabricación de productos minerales no metálicos n.c.p (1), Fabricación de productos primarios de metales preciosos y metales no ferrosos (1), Fabricación de sustancias químicas básicas, de abonos y compuestos de nitrógeno y de plásticos y caucho sintético en formas primarias (1), Reparación de productos elaborados de metal, maquinaria y equipo (1)"
21,Orellana,11,35,12.0,"$423,591.42","$11,241.57","Micro: 51.43%, Pequeña: 34.29%, Mediana: 14.29%",N: 100.00%,"Alquiler de vehículos automotores (11), Alquiler de otros tipos de maquinaria, equipo y bienes tangibles (10), Actividades de seguridad privada (5), Actividades de agencias de viajes y operadores turísticos (3), Actividades de limpieza (3), Actividades combinadas de apoyo a instalaciones (2), Otros servicios de reservas y actividades conexas (1)"
22,Santa Elena,32,33,4.5,"$2,196,846.38","$19,588.64",Mediana: 100.00%,A: 100.00%,"Acuicultura (22), Pesca (7), Cultivo de plantas perennes (2), Cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta) (2)"
4,Chimborazo,11,48,15.5,"$266,314.28","$7,668.31","Micro: 93.75%, Pequeña: 4.17%, Grande: 2.08%",N: 100.00%,"Actividades de agencias de viajes y operadores turísticos (29), Actividades de seguridad privada (8), Actividades de servicios de sistemas de seguridad (4), Actividades de servicios de apoyo a las empresas n.c.p (3), Actividades combinadas de apoyo a instalaciones (2), Alquiler de vehículos automotores (2)"


In [206]:
grouped_df.to_csv(
    'Datasets Procesados\\Resultados\\LQ_Detalles_Clusters_dbscan.csv', 
    sep=';',               # Separador de columnas
    decimal=',',           # Separador decimal
    index=False,           # Excluir índice
    encoding='utf-8-sig'   # Codificación compatible con Excel y Power BI
)